# Chemical equilibrium with fixed volume and internal energy

Let's consider the combustion of CH<sub>4</sub> in a rigid and adiabatic chamber of 10 cm<sup>3</sup>. Therefore, we will solve a chemical equilibrium problem in which we specify **volume** and **internal energy** as given properties at the equilibrium state.

In [2]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

Next, we create an initial chemical state for this system in which CH<sub>4</sub> and O<sub>2</sub> exist in a 10 cm<sup>3</sup> volume with mole fractions 0.75 and 0.25 of 1 mol, respectively. This is followed by **scaling the volume of the chemical state** to the desired 10 cm<sup>3</sup>.

In [3]:
state = ChemicalState(system)
state.temperature(25.0, "celsius")
state.pressure(1.0, "bar")
state.set("CH4", 0.75, "mol")  # 75% of 1 mol
state.set("O2",  0.25, "mol")  # 25% of 1 mol
state.scaleVolume(10.0, "cm3")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   298.1500 |    K |
| Pressure        |     1.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 1.2102e-03 |  mol |
| :: C            | 3.0255e-04 |  mol |
| :: O            | 2.0170e-04 |  mol |
| Species Amount: |            |      |
| :: CH4          | 3.0255e-04 |  mol |
| :: O2           | 1.0085e-04 |  mol |
| :: CO2          | 4.0340e-20 |  mol |
| :: CO           | 4.0340e-20 |  mol |
| :: H2O          | 4.0340e-20 |  mol |
| :: H2           | 4.0340e-20 |  mol |
+-----------------+------------+------+


Let's check the volume and internal energy in this initial chemical state using [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) class.

In [4]:
props = ChemicalProps(state)

V0 = props.volume()          # the initial volume of the gases
U0 = props.internalEnergy()  # the initial internal energy of the gases

print("Initial volume of the gases is", V0, "m3")
print("Initial internal energy of the gases is", U0, "J")

Initial volume of the gases is 1e-05 m3
Initial internal energy of the gases is -23.5698 J


Considered chemical equilibrium problem needs to impose constraints on the following properties:

* volume and
* internal energy.

Below, we create an [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) object to reflect these specifications for the equilibrium constraints, which is then used to create our [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) object and specify the values of volume and internal energy in the [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html) object (which should be valued at the initial state, `V0` and `U0`, respectively).

In [5]:
specs = EquilibriumSpecs(system)
specs.volume()
specs.internalEnergy()

solver = EquilibriumSolver(specs)

conditions = EquilibriumConditions(specs)
conditions.volume(V0)
conditions.internalEnergy(U0)

conditions.setLowerBoundTemperature(25.0, "celsius")
conditions.setUpperBoundTemperature(2000.0, "celsius")

conditions.setLowerBoundPressure(1.0, "bar")
conditions.setUpperBoundPressure(1000.0, "bar")

> **Tip**: It's good to set lower and upper bounds for temperature and pressure when solving chemical equilibrium problems in which these properties are unknown. This helps to avoid them being set to negative or extremely large values, causing errors in the algorithm.

Finally, we can  perform the equilibrium calculation:

In [6]:
solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1080.3867 |    K |
| Pressure        |     5.8362 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 1.2102e-03 |  mol |
| :: C            | 3.0255e-04 |  mol |
| :: O            | 2.0170e-04 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.2897e-04 |  mol |
| :: O2           | 1.0000e-16 |  mol |
| :: CO2          | 9.6874e-06 |  mol |
| :: CO           | 1.6389e-04 |  mol |
| :: H2O          | 1.8430e-05 |  mol |
| :: H2           | 3.2873e-04 |  mol |
+-----------------+------------+------+


Note that O<sub>2</sub> was entirely consumed in the process. Its amount is not zero because Reaktoro's equilibrium solver has a default lower bound of 1e-16 moles for the species amounts.

> **Tip**: To change this default lower bound for species amounts, [EquilibriumOptions](https://reaktoro.org/api/structReaktoro_1_1EquilibriumOptions.html) class can be used:
>
> ~~~python
> options = EquilibriumOptions()
> options.epsilon = 1e-30
>
> solver = EquilibriumSolver(specs)
> solver.setOptions(options)
> ~~~

**TASK 1**: verify that equilibrium state has volume and internal energy equal to `V0` and `U0`, respectively.

In [7]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

V = state.props().volume()
U = state.props().internalEnergy()

print("Volume at initial state:", V0, "m3")
print("Volume at final state:", V, "m3")

print("Internal energy at initial state:", U0, "J")
print("Internal energy at final state:", U, "J")

Volume at initial state: 1e-05 m3
Volume at final state: 1e-05 m3
Internal energy at initial state: -23.5698 J
Internal energy at final state: -23.5698 J


**TASK 2**: check the final temperature and pressure when burning CH<sub>4</sub> in that rigid and adiabatic chamber with our given initial conditions.

In [8]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

T = units.convert(state.temperature(), "K", "degC")  # convert from K to °C
P = units.convert(state.pressure(), "Pa", "bar")  # convert from Pa to bar

print("Temperature at final state:", T, "°C")
print("Pressure at final state:", P, "bar")

Temperature at final state: 807.237 °C
Pressure at final state: 5.83621 bar
